In [1]:
import discretizer
from privacy_mechanism import LapU
import torch
device = torch.device("cpu")

In [3]:
a=torch.tensor([[1,2,3],[1,2,3]])

In [2]:
laputest = LapU(device)

In [32]:
laputest.privatize()